Implementation of Connect 4 AI with inspirations from the following:
    http://blog.gamesolver.org/solving-connect-four/01-introduction/
    https://github.com/nwestbury/pyConnect4
    https://towardsdatascience.com/creating-the-perfect-connect-four-ai-bot-c165115557b0


In [61]:
import numpy
from kaggle_environments import make

# env = make("connectx", debug=True)
# # None indicates which agent will be manually played.
# env.run(["random", "random"])
# env.render(mode="ipython", width=500, height=450)

In [62]:
# from submission import my_agent

In [63]:
env = make("connectx", debug=True)
# None indicates which agent will be manually played.
# env.run(["negamax", "negamax"])
# env.render(mode="ipython", width=500, height=450)

In [64]:
def my_agent3(obs, config):
    import numpy as np
    import random
    import time
    
    
    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)

    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows
    
    # Helper function for minimax: calculates value of heuristic for grid
    def get_heuristic(grid, mark, config):
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_threes_opp = count_windows(grid, 3, mark%2+1, config)
        num_fours_opp = count_windows(grid, 4, mark%2+1, config)        
#         score = 1e2*num_threes - 1e2*num_threes_opp - 1e9*num_fours_opp + 1e9*num_fours
        score = 10 * num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
        return score
    
    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(grid, col, mark, config, nsteps):
        next_grid = drop_piece(grid, col, mark, config)        
        score = -negamax(next_grid, nsteps - 1, mark%2+1, config)
        return score

    # Helper function for minimax: checks if agent or opponent has four in a row in the window
    def is_terminal_window(window, config):
        return window.count(1) == config.inarow or window.count(2) == config.inarow

    # Helper function for minimax: checks if game has ended
    def is_terminal_node(grid, config):
        # Check for draw 
        if list(grid[0, :]).count(0) == 0:
            return True
        
        # Check for win: horizontal, vertical, or diagonal
        # horizontal 
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if is_terminal_window(window, config):
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if is_terminal_window(window, config):
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if is_terminal_window(window, config):
                    return True
        return False
    

    def negamax(node, depth, mark, config, alpha=-np.Inf, beta=np.Inf):
        """
        function negamax(node, depth, α, β, color) is
        if depth = 0 or node is a terminal node then
            return color × the heuristic value of node

        childNodes := generateMoves(node)
        childNodes := orderMoves(childNodes)
        value := −∞
        foreach child in childNodes do
            value := max(value, −negamax(child, depth − 1, −β, −α, −color))
            α := max(α, value)
            if α ≥ β then
                break (* cut-off *)
        return alpha
        """
        global pos_count
        pos_count += 1
        is_terminal = is_terminal_node(node, config)
        
        if depth == 0 or is_terminal:
            return get_heuristic(node, mark, config)
#         valid_moves = [c for c in range(config.columns) if node[0][c] == 0]  
        valid_moves = [column_order[col] for col in range(config.columns) if obs.board[column_order[col]] == 0]
#         valid_moves := orderMoves(valid_moves)
        value = -np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            value = max(value, -negamax(child, depth - 1, mark%2+1, config, -beta, -alpha))
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return alpha
    
    
#     def iterative_deepening(grid, obs.mark, config, timeout):
#         best_col, best_score = -1, -np.Inf
#         valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
#         for depth in range(3, 50):
#             for col in valid_moves:
#                 next_grid = drop_piece(grid, col, mark, config)        
#                 score = -negamax(next_grid, nsteps - 1, mark%2+1, config)
#                 if not search_cutoff and score > best_score:
#                     best_col, best_score = col, best_score
#                 else: # Running out of time
#                     break
#         return best_col
        
        
        
        
  
    start = time.time()
    global pos_count
    pos_count = 0
    column_order = []
    for i in range(7):
        column_order.append(7//2 + (1 - 2* (i %2)) * (i+1)//2)
    
#     valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    # Order column from center to side
    valid_moves = [column_order[col] for col in range(config.columns) if obs.board[column_order[col]] == 0]
    # Very first move, always play the middle
    if sum(obs.board) == 0:
        return 3
    elif sum(obs.board) == 1 and 3 in valid_moves:
        return 3 # Play middle whether opponent places there or not
    
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # print(grid)
    #Get scores for each branch
    N_STEPS = 4
    
    # Use the heuristic to assign a score to each possible board in the next step
    if len(valid_moves) >= 6:
        N_STEPS=4
    if len(valid_moves) ==5:
        N_STEPS=5
    if len(valid_moves) ==4:
        N_STEPS=6
    if len(valid_moves) <=3:
        N_STEPS=10
        
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))        
    
    #Get the highest score value    
    
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]    
    col = max_cols[0] # Taking the center columns first
    end = time.time()
#     print("timeout", config.timeout)
    print("my_agent3 excecution time", (end-start), "move", col, "score", scores[col], "at depth", N_STEPS, "pos count", pos_count)
    
    return col

        

In [65]:
# Global vairable initialization for cached_positions
cached_positions = None

In [135]:
from collections import OrderedDict
 
class LRUCache:
 
    # initialising capacity
    def __init__(self, capacity: int):
        self.cache = OrderedDict()
        self.capacity = capacity
 
    # we return the value of the key
    # that is queried in O(1) and return -1 if we
    # don't find the key in out dict / cache.
    # And also move the key to the end
    # to show that it was recently used.
    def get(self, key: int) -> object:
        if key not in self.cache:
            return None
        else:
            self.cache.move_to_end(key)
            return self.cache[key]
 
    # first, we add / update the key by conventional methods.
    # And also move the key to the end to show that it was recently used.
    # But here we will also check whether the length of our
    # ordered dictionary has exceeded our capacity,
    # If so we remove the first key (least recently used)
    def put(self, key: int, value: object) -> None:
        self.cache[key] = value
        self.cache.move_to_end(key)
        if len(self.cache) > self.capacity:
            self.cache.popitem(last = False)

transposition_table = LRUCache(1000000)
# cache.put(1, (-1, 0, 7))
# cache.put(2, (-2, 5, 5))
# print(cache.get(1))

# print(cache.cache)

In [136]:
import numpy as np
from functools import lru_cache

# return a bitmask containg a single 1 corresponding to the top cel of a given column
@lru_cache(maxsize=None)
def top_mask_col(col):
    return (1 << 5) << col * 7

# return a bitmask containg a single 1 corresponding to the bottom cell of a given column
@lru_cache(maxsize=None)
def bottom_mask_col(col):
    return 1 << col * 7

# return a bitmask 1 on all the cells of a given column
@lru_cache(maxsize=None)
def column_mask(col):
    return ((1 << 6) - 1) << col * 7

@lru_cache(maxsize=None)
def has_won(position):
    # Horizontal check
    m = position & (position >> 7)
    if m & (m >> 14):
        return True
    # Diagonal \
    m = position & (position >> 6)
    if m & (m >> 12):
        return True
    # Diagonal /
    m = position & (position >> 8)
    if m & (m >> 16):
        return True
    # Vertical
    m = position & (position >> 1)
    if m & (m >> 2):
        return True
    # Nothing found
    return False

@lru_cache(maxsize=None)
def evaluate3(oppBoard, myBoard):
    """
    Returns the number of possible 3 in a rows in bitboard format.
    Running time: O(1)
    http://www.gamedev.net/topic/596955-trying-bit-boards-for-connect-4/
    """
    inverseBoard = ~(myBoard | oppBoard)
    rShift7MyBoard = myBoard >> 7
    lShift7MyBoard = myBoard << 7
    rShift14MyBoard = myBoard >> 14
    lShit14MyBoard = myBoard << 14
    rShift16MyBoard = myBoard >> 16
    lShift16MyBoard = myBoard << 16
    rShift8MyBoard = myBoard >> 8
    lShift8MyBoard = myBoard << 8
    rShift6MyBoard = myBoard >> 6
    lShift6MyBoard = myBoard << 6
    rShift12MyBoard = myBoard >> 12
    lShift12MyBoard = myBoard << 12

    # check _XXX and XXX_ horizontal
    result = inverseBoard & rShift7MyBoard & rShift14MyBoard\
        & (myBoard >> 21)

    result |= inverseBoard & rShift7MyBoard & rShift14MyBoard\
        & lShift7MyBoard

    result |= inverseBoard & rShift7MyBoard & lShift7MyBoard\
        & lShit14MyBoard

    result |= inverseBoard & lShift7MyBoard & lShit14MyBoard\
        & (myBoard << 21)

    # check XXX_ diagonal /
    result |= inverseBoard & rShift8MyBoard & rShift16MyBoard\
        & (myBoard >> 24)

    result |= inverseBoard & rShift8MyBoard & rShift16MyBoard\
        & lShift8MyBoard

    result |= inverseBoard & rShift8MyBoard & lShift8MyBoard\
        & lShift16MyBoard

    result |= inverseBoard & lShift8MyBoard & lShift16MyBoard\
        & (myBoard << 24)

    # check _XXX diagonal \
    result |= inverseBoard & rShift6MyBoard & rShift12MyBoard\
        & (myBoard >> 18)

    result |= inverseBoard & rShift6MyBoard & rShift12MyBoard\
        & lShift6MyBoard

    result |= inverseBoard & rShift6MyBoard & lShift6MyBoard\
        & lShift12MyBoard

    result |= inverseBoard & lShift6MyBoard & lShift12MyBoard\
        & (myBoard << 18)

    # check for _XXX vertical
    result |= inverseBoard & (myBoard << 1) & (myBoard << 2)\
        & (myBoard << 3)

    return result

@lru_cache(maxsize=None)
def evaluate2(myBoard, oppBoard):
    """
    Returns the number of possible 2 in a rows in bitboard format.
    Running time: O(1)
    """
    inverseBoard = ~(myBoard | oppBoard)
    rShift7MyBoard = myBoard >> 7
    rShift14MyBoard = myBoard >> 14
    lShift7MyBoard = myBoard << 7
    lShift14MyBoard = myBoard << 14
    rShift8MyBoard = myBoard >> 8
    lShift8MyBoard = myBoard << 8
    lShift16MyBoard = myBoard << 16
    rShift16MyBoard = myBoard >> 16
    rShift6MyBoard = myBoard >> 6
    lShift6MyBoard = myBoard << 6
    rShift12MyBoard = myBoard >> 12
    lShift12MyBoard = myBoard << 12

    # check for _XX
    result = inverseBoard & rShift7MyBoard & rShift14MyBoard
    result |= inverseBoard & rShift7MyBoard & rShift14MyBoard
    result |= inverseBoard & rShift7MyBoard & lShift7MyBoard

    # check for XX_
    result |= inverseBoard & lShift7MyBoard & lShift14MyBoard

    # check for XX / diagonal
    result |= inverseBoard & lShift8MyBoard & lShift16MyBoard

    result |= inverseBoard & rShift8MyBoard & rShift16MyBoard
    result |= inverseBoard & rShift8MyBoard & rShift16MyBoard
    result |= inverseBoard & rShift8MyBoard & lShift8MyBoard

    # check for XX \ diagonal
    result |= inverseBoard & rShift6MyBoard & rShift12MyBoard
    result |= inverseBoard & rShift6MyBoard & rShift12MyBoard
    result |= inverseBoard & rShift6MyBoard & lShift6MyBoard
    result |= inverseBoard & lShift6MyBoard & lShift12MyBoard

    # check for _XX vertical
    result |= inverseBoard & (myBoard << 1) & (myBoard << 2) \
        & (myBoard << 2)

    return result

@lru_cache(maxsize=None)
def evaluate1(oppBoard, myBoard):
    """
    Returns the number of possible 1 in a rows in bitboard format.
    Running time: O(1)
    Diagonals are skipped since they are worthless.
    """
    inverseBoard = ~(myBoard | oppBoard)
    # check for _X
    result = inverseBoard & (myBoard >> 7)

    # check for X_
    result |= inverseBoard & (myBoard << 7)

    # check for _X vertical
    result |= inverseBoard & (myBoard << 1)

    return result

@lru_cache(maxsize=None)
def bitboardBits(i):
    """"
    Returns the number of bits in a bitboard (7x6).
    Running time: O(1)
    Help from: http://stackoverflow.com/q/9829578/1524592
    """
    i = i & 0xFDFBF7EFDFBF  # magic number to mask to only legal bitboard
    # positions (bits 0-5, 7-12, 14-19, 21-26, 28-33, 35-40, 42-47)
    i = (i & 0x5555555555555555) + ((i & 0xAAAAAAAAAAAAAAAA) >> 1)
    i = (i & 0x3333333333333333) + ((i & 0xCCCCCCCCCCCCCCCC) >> 2)
    i = (i & 0x0F0F0F0F0F0F0F0F) + ((i & 0xF0F0F0F0F0F0F0F0) >> 4)
    i = (i & 0x00FF00FF00FF00FF) + ((i & 0xFF00FF00FF00FF00) >> 8)
    i = (i & 0x0000FFFF0000FFFF) + ((i & 0xFFFF0000FFFF0000) >> 16)
    i = (i & 0x00000000FFFFFFFF) + ((i & 0xFFFFFFFF00000000) >> 32)

    return i

@lru_cache(maxsize=None)
def evaluate_position(oppBoard, myBoard):
    """
    Returns cost of each board configuration.
    winning is a winning move
    blocking is a blocking move
    Running time: O(7n)
    """
    winReward = 9999999
    OppCost3Row = 1000
    MyCost3Row = 3000
    OppCost2Row = 500
    MyCost2Row = 500
    OppCost1Row = 100
    MyCost1Row = 100

    if has_won(oppBoard):
        return -winReward
    elif has_won(myBoard):
        return winReward
    # elif self.has_drawn():
    #     return 0  # draw score

    get3Win = evaluate3(oppBoard, myBoard)
    winning3 = bitboardBits(get3Win) * MyCost3Row

    get3Block = evaluate3(myBoard, oppBoard)
    blocking3 = bitboardBits(get3Block) * -OppCost3Row

    get2Win = evaluate2(oppBoard, myBoard)
    winning2 = bitboardBits(get2Win) * MyCost2Row

    get2Block = evaluate2(myBoard, oppBoard)
    blocking2 = bitboardBits(get2Block) * -OppCost2Row

    get1Win = evaluate1(oppBoard, myBoard)
    winning1 = bitboardBits(get1Win) * MyCost1Row

    get1Block = evaluate1(myBoard, oppBoard)
    blocking1 = bitboardBits(get1Block) * -OppCost1Row

    return winning3 + blocking3 + winning2 + blocking2\
        + winning1 + blocking1

@lru_cache(maxsize=None)
def compute_winning_position(position, mask):
    HEIGHT = 6
    # vertical
    r = (position << 1) & (position << 2) & (position << 3)

    # horizontal
    p = (position << (6+1)) & (position << 2*(6+1))
    r |= p & (position << 3*(6+1))
    r |= p & (position >> (6+1))
    p = (position >> (6+1)) & (position >> 2*(6+1))
    r |= p & (position << (6+1))
    r |= p & (position >> 3*(6+1))

    # diagonal 1
    p = (position << HEIGHT) & (position << 2*HEIGHT)
    r |= p & (position << 3*HEIGHT)
    r |= p & (position >> HEIGHT)
    p = (position >> HEIGHT) & (position >> 2*HEIGHT)
    r |= p & (position << HEIGHT)
    r |= p & (position >> 3*HEIGHT)

    # diagonal 2
    p = (position << (HEIGHT+2)) & (position << 2*(HEIGHT+2))
    r |= p & (position << 3*(HEIGHT+2))
    r |= p & (position >> (HEIGHT+2))
    p = (position >> (HEIGHT+2)) & (position >> 2*(HEIGHT+2))
    r |= p & (position << (HEIGHT+2))
    r |= p & (position >> 3*(HEIGHT+2))

    return r & (279258638311359 ^ mask)

class Position(object):
    WIDTH = 7
    HEIGHT = 6
    bottom_mask = 4432676798593
    board_mask = 279258638311359

    def __init__(self, position, mask):
        self.position = position
        self.mask = mask
        self.moves = bitboardBits(self.mask)   #TODO: do this only once, not when creating child positions as we already have moves by then

    @classmethod
    def get_position_mask_bitmap(self, grid, mark):
        # print("grid", grid)
        # print("mark", mark)
        position, mask = '', ''
        # Start with right-most column
        for j in range(6, -1, -1):
            # Add 0-bits to sentinel
            mask += '0'
            position += '0'
            # Start with bottom row
            for i in range(0, 6):
                mask += ['0', '1'][int(grid[i][j] != 0)]
                position += ['0', '1'][int(grid[i][j] == mark)]
        return int(position, 2), int(mask, 2)

    def key(self):
        return self.position + self.mask

    def partial_key3(self, key, col):
        for 

    def can_play(self, col):
        return (self.mask & top_mask_col(col)) == 0

    def play(self, col):
        self.position ^= self.mask
        self.mask |= self.mask + bottom_mask_col(col)
        self.moves += 1

    def is_winning_move(self, col):
        pos = self.position
        pos |= (self.mask + bottom_mask_col(col)) & column_mask(col)
        return self.has_won(pos)

    def has_drawn(self):
        """
        If the board has all of its valid slots filled, then it is a draw.
        We mask the board to a bitboard with all positions filled
        (0xFDFBF7EFDFBF) and if all the bits are active, it is a draw.
        """
        # return (self.position & 0xFDFBF7EFDFBF) == 0xFDFBF7EFDFBF
        return self.moves == 42

    

    def pretty_print(self):
        print("position", self.position)
        print("mask", self.mask)
        opp_position = self.position ^ self.mask
        print("board     position  mask      key       bottom")
        print("          0000000   0000000")
        for i in range(5, -1, -1):  # iterate backwards from 5 to 0
            board_row = "".join("x" if (self.position >> i+k*7) & 1 == 1
                                else "o" if (opp_position >> i+k*7) & 1 == 1 else "." for k in range(7))
            pos_row = "".join(str((self.position >> i+j*7) & 1)
                            for j in range(7))
            mask_row = "".join(str((self.mask >> i+j*7) & 1) for j in range(7))
            print(board_row + "   " + pos_row + "   " + mask_row)

    

    def possible(self):
        return (self.mask + self.bottom_mask) & self.board_mask

    # Return a bitmask of the possible winning positions for the opponent
    def opponent_winning_position(self):
        return compute_winning_position(self.position ^ self.mask, self.mask)

    # Return a bitmask of the possible winning positions for the current player
    def winning_position(self):
        return compute_winning_position(self.position, self.mask)

    

    # Indicates whether the current player wins by playing a given column.
    # This function should never be called on a non-playable column.
    def is_winning_move2(self, col):
        return self.winning_position() & self.possible() & column_mask(col)


    def can_win_next(self):
        return self.winning_position() & self.possible()

    # Return a bitmap of all the possible next moves the do not lose in one turn.
    # A losing move is a move leaving the possibility for the opponent to win directly.
    def possible_non_losing_moves(self):
        possible_mask = self.possible()
        opponent_win = self.opponent_winning_position()
        forced_moves = possible_mask & opponent_win
        if forced_moves:
            if forced_moves & (forced_moves - 1):
                return 0
            else:
                possible_mask = forced_moves
        return possible_mask & ~(opponent_win >> 1)


class Solver(object):
    cache = None
    def __init__(self):
        self.node_count = 0
        self.column_order = []
        for i in range(7):
            self.column_order.append(7 // 2 + (1 - 2 * (i % 2)) * (i + 1) // 2)

        global cached_positions
        if cached_positions == None:
            print("initializing cached_positions...")
            cached_positions = dict()
            with open("./connect-4.data") as f:
                count = 0
                for line in f:
                    l = line.split(",")
                    s = l[-1].strip("\n")
                    l = [1 if x == 'x' else 2 if x == 'o' else 0 for x in l[:-1]]
                    grid = np.asarray(l).reshape(7, 6)
                    grid = np.rot90(grid)
                    position, mask = Position.get_position_mask_bitmap(grid, 1)
                    p = Position(position, mask)
                    # p.pretty_print()
                    # print(s)
                    count += 1
                    cached_positions[position + mask] = 1 if s == 'win' else -1 if s == "loss" else 0  
        self.cache = cached_positions   

    # def negamax(self, p, alpha, beta):
    #     self.node_count += 1
    #     if p.has_drawn(): # check for draw game
    #         return 0

    #     for x in range(7): # check if current player can win next move
    #         if p.can_play(x) and p.is_winning_move(x):
    #             return
    #     return alpha

    def negamax(self, p: Position, depth, mark, config, alpha=-np.Inf, beta=np.Inf):
        """
        function negamax(node, depth, α, β, color) is
            alphaOrig := α

            (* Transposition Table Lookup; node is the lookup key for ttEntry *)
            ttEntry := transpositionTableLookup(node)
            if ttEntry is valid and ttEntry.depth ≥ depth then
                if ttEntry.flag = EXACT then
                    return ttEntry.value
                else if ttEntry.flag = LOWERBOUND then
                    α := max(α, ttEntry.value)
                else if ttEntry.flag = UPPERBOUND then
                    β := min(β, ttEntry.value)

                if α ≥ β then
                    return ttEntry.value

            if depth = 0 or node is a terminal node then
                return color × the heuristic value of node

            childNodes := generateMoves(node)
            childNodes := orderMoves(childNodes)
            value := −∞
            for each child in childNodes do
                value := max(value, −negamax(child, depth − 1, −β, −α, −color))
                α := max(α, value)
                if α ≥ β then
                    break

        """
        # assert(alpha < beta)
        # assert(not p.can_win_next())
        # p.pretty_print()
        self.node_count += 1
        alpha_orig = alpha

        # Check 8 ply cache first
        if p.moves == 8 and p.position + p.mask in self.cache:
            if self.cache[p.position + p.mask] == 1:
                # print("at depth", depth, "matched to a winning position for player 1")
                return 9999999
            elif self.cache[p.position + p.mask] == -1:
                # print("at depth", depth, "matched to a losing position for player 1")
                return -9999999
            else:
                # print("at depth", depth, "matched to a draw position for player 1")
                return 0

        # Transposition table lookup
        tt_entry = transposition_table.get(p.key())
        if tt_entry != None and tt_entry[2] >= depth:
            if tt_entry[1] == 0: # Exact
                return tt_entry[0]
            elif tt_entry[1] == 1: # lowerbound
                alpha = max(alpha, tt_entry[0])
            elif tt_entry[1] == 2: # upperbound
                beta = min(beta, tt_entry[0])

            if alpha >= beta:
                return tt_entry[0]

    
        # Opponent must've won, as the current position was provided from the previous move
        is_terminal = (p.has_drawn() or has_won(p.position))

        if depth == 0 or is_terminal:
            return -evaluate_position(p.position, p.position ^ p.mask)

        if p.can_win_next():
            return 9999999

        next = p.possible_non_losing_moves()
        if next == 0: # no possible non losing move, opponent wins next move
            # p.pretty_print()
            # print("opponent wins next")
            return -9999999

        if p.moves >= 40: # check for draw game
            return 0

        value = -np.Inf
        # for col in valid_moves:
        for col in range(7):
            if next & column_mask(self.column_order[col]) > 0:
                child = Position(p.position, p.mask)
                # print("Playing ", col, "with mark", mark)
                child.play(self.column_order[col])
                
                value = max(value, -self.negamax(child, depth -
                            1, mark % 2+1, config, -beta, -alpha))
                alpha = max(alpha, value)
                if alpha >= beta:
                    break

        # Transposition table store
        if value <= alpha_orig:
            transposition_table.put(p.key(), (value, 2, depth))
        elif value >= beta:
            transposition_table.put(p.key(), (value, 1, depth))
        else:
            transposition_table.put(p.key(), (value, 0, depth))
                
        return value

    def negamax2(self, p: Position, depth, mark, config, alpha=-np.Inf, beta=np.Inf):
        """
        function negamax(node, depth, α, β, color) is
        if depth = 0 or node is a terminal node then
            return color × the heuristic value of node

        childNodes := generateMoves(node)
        childNodes := orderMoves(childNodes)
        value := −∞
        foreach child in childNodes do
            value := max(value, −negamax(child, depth − 1, −β, −α, −color))
            α := max(α, value)
            if α ≥ β then
                break (* cut-off *)
        return alpha
        """
        # assert(alpha < beta)
        # assert(not p.can_win_next())
        # p.pretty_print()
        self.node_count += 1
        # Opponent must've won, as the current position was provided from the previous move
        is_terminal = (p.has_drawn() or has_won(p.position))

        if depth == 0 or is_terminal:
            return -evaluate_position(p.position, p.position ^ p.mask)

        if p.can_win_next():
            return 9999999

        next = p.possible_non_losing_moves()
        if next == 0: # no possible non losing move, opponent wins next move
            # p.pretty_print()
            # print("opponent wins next")
            return -9999999

        if p.moves >= 40: # check for draw game
            return 0

        if p.moves == 8 and p.position + p.mask in self.cache:

            if self.cache[p.position + p.mask] == 1:
                # print("at depth", depth, "matched to a winning position for player 1")
                return 9999999
            elif self.cache[p.position + p.mask] == -1:
                # print("at depth", depth, "matched to a losing position for player 1")
                return -9999999
            else:
                # print("at depth", depth, "matched to a draw position for player 1")
                return 0
        # valid_moves = [self.column_order[x]
        #                for x in range(7) if p.can_play(self.column_order[x])]
        # min = -(40 - p.moves) / 2	# lower bound of score as opponent cannot win next move
        # if alpha < min:
        #     alpha = min                   # there is no need to keep beta above our max possible score.
        #     if(alpha >= beta):
        #          return alpha   #  prune the exploration if the [alpha;beta] window is empty.
        

        # max = (41 - p.moves)/2	   # upper bound of our score as we cannot win immediately
        # # if(int val = transTable.get(P.key()))
        # # max = val + Position::MIN_SCORE - 1;

        # if beta > max: 
        #     beta = max                     # there is no need to keep beta above our max possible score.
        # if alpha >= beta:
        #      return beta;  # prune the exploration if the [alpha;beta] window is empty.
      
        value = -np.Inf
        # for col in valid_moves:
        for col in range(7):
            if next & column_mask(self.column_order[col]) > 0:
                child = Position(p.position, p.mask)
                # print("Playing ", col, "with mark", mark)
                child.play(self.column_order[col])
                # child.pretty_print()

                # value = max(value, -self.negamax(child, depth -
                #             1, mark % 2+1, config, -beta, -alpha))
                # alpha = max(alpha, value)
                # if alpha >= beta:
                #     break

                score = -self.negamax(child, depth -
                            1, mark % 2+1, config, -beta, -alpha)
                if score >= beta:
                    return score
                if score >= alpha:
                    alpha = score
                
        return alpha

    # Uses minimax to calculate value of dropping piece in selected column
    def score_move(self, p: Position, col, mark, config, nsteps):
        
        next_grid = Position(p.position, p.mask)
        next_grid.play(col)
        score = -self.negamax(next_grid, nsteps - 1, mark % 2+1, config)
        return score

    # def forced_moves(self, p:Position, valid_moves):
    #     """
    #     If placing a token can win immediately, return that column.
    #     Otherwise, if you can block your opponent immediately, return
    #     one of those column(s).
    #     """

    #     myBoard = p.position
    #     oppBoard = p.position ^ p.mask
    #     possibleBits = self.get_legal_locations(myBoard | oppBoard)

    #     forcedCols = []  # cols needed to block your opponent if you cannot win
    #     for colbitTuple in possibleBits:
    #         tempMyBoard = self.setNthBit(myBoard, colbitTuple[1])
    #         tempOppBoard = self.setNthBit(oppBoard, colbitTuple[1])

    #         if board.hasWon(tempMyBoard):
    #             return colbitTuple[0]
    #         elif board.hasWon(tempOppBoard):
    #             forcedCols.append(colbitTuple[0])

    #     if forcedCols:
    #         return forcedCols[0]
    #     return -1

    # def solve(self, p, weak=False):
    #     self.node_count = 0
    #     if weak:
    #         return self.negamax(p, -1, 1)
    #     else:
    #         return self.negamax(p, -7 * 6 //2, 7*6//2)

# grid = [
#  [0,0,0,0,0,0,0],
#  [0,0,0,0,0,0,0],
#  [0,0,0,0,0,0,0],
#  [0,0,0,0,0,0,0],
#  [0,0,0,0,0,0,0],
#  [0,0,0,2,1,1,2]
#  ]

# position, mask = Position.get_position_mask_bitmap(grid, 1)
# p = Position(position, mask)
# # p = Position(0b0000000000000100000010000000000000000000000000000, 0b0000001000000100000010000001000000000000000000000)
# print("top_mask", format(p.top_mask(1), "064b"))
# print("bottom_mask", format(p.bottom_mask(1), "064b"))
# print("column_mask", format(p.column_mask(1), "064b"))
# print("can play", p.can_play(3))
# p.pretty_print()
# p.play(3)
# print("position after play", format(p.position, "049b"))
# p.pretty_print()
# p.play(3)
# p.play(3)
# p.play(3)
# p.play(3)
# print("can now play", p.can_play(3))
# p.play(2)
# p.pretty_print()
# print("has_drawn", p.has_drawn())

In [160]:
def my_agent(obs, config):
    import numpy as np
    import random
    import time
    import cProfile, pstats, io
    from pstats import SortKey
    
    
    # with cProfile.Profile() as pr:

    start = time.time()
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # print(grid)
    
    # Very first move, always play the middle
    if sum(obs.board) == 0:
        return 3
    elif sum(obs.board) == 1 and grid[0, 3] == 0:
        return 3 # Play middle whether opponent places there or not

    position, mask = Position.get_position_mask_bitmap(grid, obs.mark)
    # print(position, mask)
    p = Position(position, mask)
    solver = Solver()
    valid_moves = [solver.column_order[x] for x in range(7) if p.can_play(solver.column_order[x])]
    # If there is a winning move, return it now!
    if p.can_win_next():
        # print("can win next", p.position, p.mask)
        # p.pretty_print()
        for col in valid_moves:
            if p.is_winning_move2(col) > 0:
                print("my_agent_new winning move", col)
                return col

    # Use the heuristic to assign a score to each possible board in the next step
    if len(valid_moves) >= 6:
        N_STEPS=10
    if len(valid_moves) ==5:
        N_STEPS=12
    if len(valid_moves) ==4:
        N_STEPS= 16
    if len(valid_moves) <=3:
        N_STEPS= 20

    # if p.moves == 29:
    #     p.pretty_print()
    scores = dict(zip(valid_moves, [solver.score_move(p, col, obs.mark, config, N_STEPS) for col in valid_moves]))   
    
    #Get the highest score value    
    
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]    
    # col = random.choice(max_cols) # Taking the center columns first
    col = max_cols[0]
    end = time.time()
    print("my_agent_new move #", p.moves+1, "time", (end-start), "move", col, "score", scores[col], "at depth", N_STEPS, "pos count", solver.node_count)
        # print(evaluate_position.cache_info())
    # s = io.StringIO()
    # # sortby = SortKey.CUMULATIVE, SortKey.PCALLS
    # ps = pstats.Stats(pr, stream=s).sort_stats(SortKey.PCALLS)
    # ps.dump_stats("./profile.stats")
    # # pr.print_stats()
    # ps.print_stats(50)
    # print(s.getvalue())
    
    return col

# grid = [[0, 1, 0, 2, 2, 1, 0],
#  [0, 2, 0, 2, 1, 2, 0],
#  [0, 1, 0, 2, 1, 1, 0],
#  [0, 2, 0, 1, 2, 2, 2],
#  [0, 1, 2, 2, 1, 1, 1],
#  [2, 1, 2, 1, 1, 1, 2]]
grid = [[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0]]
# grid = [[0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 1, 0, 0, 0],
#  [0, 0, 0, 2, 0, 0, 0],
#  [0, 0, 2, 1, 0, 0, 0]]
grid = [
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0]]


# board = "b,b,b,b,b,b,b,b,b,b,b,b,x,b,b,b,b,b,x,o,x,o,x,o,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,win"
# l = board.split(",")
# l = [1 if x == 'x' else 2 if x == 'o' else 0 for x in l[:-1]]
# grid = np.asarray(l).reshape(7, 6)
# print(grid)
# grid = np.rot90(grid)
# print(grid)

# grid = [
#  [0, 1, 2, 2, 0, 1, 0],
#  [0, 2, 2, 2, 0, 2, 0],
#  [2, 1, 2, 1, 0, 1, 0],
#  [2, 2, 1, 1, 0, 2, 0],
#  [1, 2, 1, 2, 0, 1, 0],
#  [1, 1, 2, 1, 2, 1, 1]]

# grid = [
#  [2, 1, 1, 1, 2, 2, 0],
#  [1, 2, 1, 1, 1, 2, 1],
#  [1, 1, 1, 2, 2, 1, 1],
#  [2, 2, 2, 1, 2, 2, 2],
#  [2, 1, 1, 2, 1, 1, 1],
#  [2, 2, 2, 1, 1, 2, 2]]

# position, mask = Position.get_position_mask_bitmap(grid, 2)

# solver = Solver()
# p = Position(position, mask)

# print("bit number", p.bitboardBits(p.mask))
# print("has drawn", p.has_drawn())
# print("winning pos", p.is_winning_move2(6))
# print("possible non-losing moves", p.possible_non_losing_moves())

# valid_moves = [solver.column_order[x] for x in range(7) if p.can_play(solver.column_order[x])]
# print(valid_moves)
# scores = dict(zip(valid_moves, [solver.score_move(p, col, 1, 0, 20) for col in valid_moves]))   
# print(scores)
# max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]    
# print(max_cols)
# score = solver.score_move(p, 6, 1, 0, 12)
# print(score)
# p2 = Position(1830214572695, 6579754622911)
# p2.pretty_print()
# print("possible non-losing moves", p2.possible_non_losing_moves())
# p3 = Position(34911322241, 4433243259267)
# p3.pretty_print()
# print(p3.is_winning_move(5))
# print(p3.is_winning_move2(5))

In [33]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(my_agent, "./submission.py")

<function my_agent at 0x000001FBE548FF70> written to ./submission.py


In [161]:
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx", debug=True)

# Two random agents play one game round
# env.run([my_agent, "negamax"])
# env.run(["negamax", my_agent])
env.run([my_agent3, my_agent])
# env.run([my_agent3, "negamax"])
# env.run([my_agent, my_agent3])

# Show the game
env.render(mode="ipython")

my_agent3 excecution time 0.792686939239502 move 3 score 0.0 at depth 4 pos count 532
my_agent_new move # 4 time 0.0002300739288330078 move 4 score 800 at depth 10 pos count 7
my_agent3 excecution time 1.6047940254211426 move 3 score 0.0 at depth 4 pos count 770
my_agent_new move # 6 time 5.807048797607422 move 0 score -2500 at depth 10 pos count 217191
my_agent3 excecution time 1.0634028911590576 move 3 score 10000.0 at depth 4 pos count 705
my_agent_new move # 8 time 0.9834487438201904 move 3 score -3500 at depth 10 pos count 37285
my_agent3 excecution time 1.0784966945648193 move 4 score 0.0 at depth 4 pos count 529
my_agent_new move # 10 time 3.212790012359619 move 4 score -3400 at depth 10 pos count 121241
my_agent3 excecution time 0.8544921875 move 2 score -90.0 at depth 4 pos count 510
my_agent_new move # 12 time 1.392246961593628 move 4 score -3700 at depth 10 pos count 58548
my_agent3 excecution time 0.7667951583862305 move 4 score -90.0 at depth 4 pos count 504
my_agent_new m

In [168]:
import random as random
position = 35454536375
mask = 107238903743
solver = Solver()
p = Position(position, mask)
print("bit number", bitboardBits(p.mask))
print("winning pos", p.is_winning_move2(0))
print("possible non-losing moves", p.possible_non_losing_moves())

valid_moves = [solver.column_order[x] for x in range(7) if p.can_play(solver.column_order[x])]
print(valid_moves)
scores = dict(zip(valid_moves, [solver.score_move(p, col, 1, 0, 10) for col in valid_moves]))   
print(scores)
max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]    
col = random.choice(max_cols) # Taking the center columns first
print("move", col)


bit number 29
winning pos 0
possible non-losing moves 137438953472
[4, 1, 5, 6]
{4: -9999999, 1: -9999999, 5: -9999999, 6: -9999999}
move 4


In [163]:
def get_win_percentages(agent1, agent2, n_rounds=10):
    from kaggle_environments import make, evaluate
    import numpy as np
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # print("outcomes:", outcomes)
    print("outcomes:", outcomes)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("outcomes:", outcomes)
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [164]:
get_win_percentages(agent1=my_agent3, agent2=my_agent)

outcomes: [[-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]]
outcomes: [[-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]]
Agent 1 Win Percentage: 0.0
Agent 2 Win Percentage: 1.0
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


In [156]:
import sys
from kaggle_environments import utils, agent

out = sys.stdout
my_agent4 = utils.read_file("./submission_with_dataset.py")
my_agent5 = utils.read_file("./submission_with_dataset_10ply.py")
sys.stdout = out

# env = make("connectx", debug=True)
# env.run([agent, agent])
# env.render(mode="ipython")
# print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed..." + env.state[0].status + env.state[1].status)

In [159]:
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx", debug=True)

# Two random agents play one game round
# env.run([my_agent, "negamax"])
# env.run(["negamax", my_agent])
# env.run([my_agent3, my_agent])
# env.run([my_agent3, "negamax"])
# env.run([my_agent, my_agent3])
# env.run([my_agent5, my_agent])
env.run([my_agent, my_agent5])

# Show the game
env.render(mode="ipython")

my_agent_new move # 3 time 6.525446891784668 move 4 score 1200 at depth 11 pos count 332967
my_agent_new move # 5 time 9.342835903167725 move 2 score 1700 at depth 11 pos count 389513
my_agent_new move # 7 time 10.465687990188599 move 3 score 1600 at depth 11 pos count 294400
my_agent_new move # 9 time 14.894556045532227 move 2 score 3200 at depth 11 pos count 554052
my_agent_new move # 11 time 8.714501142501831 move 4 score 6800 at depth 11 pos count 207689
my_agent_new move # 13 time 1.003350019454956 move 6 score 9100 at depth 11 pos count 41887
my_agent_new move # 15 time 3.531385660171509 move 2 score 10400 at depth 11 pos count 151804
my_agent_new move # 17 time 1.3419189453125 move 4 score 11800 at depth 11 pos count 62307
my_agent_new move # 19 time 0.4364957809448242 move 3 score 12900 at depth 11 pos count 21769
my_agent_new move # 21 time 0.12791681289672852 move 3 score 9999999 at depth 11 pos count 6151
my_agent_new move # 23 time 0.024899005889892578 move 1 score 9999999 

In [172]:
# grid = [
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 0, 0, 0, 0],
#  [0, 0, 0, 2, 0, 0, 0],
#  [0, 0, 0, 1, 0, 0, 0],
#  [0, 0, 0, 0, 2, 1, 1],
#  [0, 0, 2, 1, 1, 2, 1]]

grid = [
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 2, 1, 0, 0, 0, 0]]

position, mask = Position.get_position_mask_bitmap(grid, 1)
print(position, mask)
solver = Solver()
p = Position(position, mask)
print("bit number", bitboardBits(p.mask))
print("winning pos", p.is_winning_move2(0))
print("possible non-losing moves", p.possible_non_losing_moves())

valid_moves = [solver.column_order[x] for x in range(7) if p.can_play(solver.column_order[x])]
print(valid_moves)
scores = dict(zip(valid_moves, [solver.score_move(p, col, 1, 0, 8) for col in valid_moves]))   
print(scores)
 

16384 16512
bit number 2
winning pos 0
possible non-losing moves 4432676815105
[3, 2, 4, 1, 5, 0, 6]
{3: -3000, 2: -9999999, 4: -1300, 1: -900, 5: 400, 0: -9999999, 6: -9999999}


In [167]:
get_win_percentages(agent1=my_agent, agent2=my_agent4)

outcomes: [[0, None], [-1, 1], [-1, 1], [-1, 1], [-1, 1]]
outcomes: [[0, None], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]]
Agent 1 Win Percentage: 0.0
Agent 2 Win Percentage: 0.9
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 1
